# NanoPyx "Codeless" Jupyter Notebook

This notebook allows you to assess quality control metrics of microscopy images such as an Error Map (NanoJ-SQUIRREL), FRC (Fourier Ring Correlation) and Decorrelation analysis.

To use this notebook you don't need to interact with any code, just run cells in order and a graphical user interface will pop-up showcasing the parameters for each step.
When running this notebook on Colab, it will automatically prompt you to connect to your google Drive, from where you can select the input data.
To save the output of this notebook, make sure to select the "save output" option. Output will be saved on the same directory as the loaded image. If using an example dataset it will be saved on the path Python is currently running (if locally running the notebook) or on Colab files (if running on Colab, files button on the left bar).
If you found this work useful for your research please consider citing our [preprint](https://www.biorxiv.org/content/10.1101/2023.08.13.553080v1)  and out Github repo [![DOI](https://zenodo.org/badge/505388398.svg)](https://zenodo.org/badge/latestdoi/505388398)

**Error Map**: Culley, S., Albrecht, D., Jacobs, C. et al. Quantitative mapping and minimization of super-resolution optical imaging artifacts. Nat Methods 15, 263–266 (2018). https://doi.org/10.1038/nmeth.4605

**FRC**: Nieuwenhuizen RP, Lidke KA, Bates M, Puig DL, Grünwald D, Stallinga S, Rieger B. Measuring image resolution in optical nanoscopy. Nat Methods. 2013 Jun;10(6):557-62. doi: 10.1038/nmeth.2448. Epub 2013 Apr 28. PMID: 23624665; PMCID: PMC4149789.

**DecorrAnalysis**: Descloux A, Grußmayer KS, Radenovic A. Parameter-free image resolution estimation based on decorrelation analysis. Nat Methods. 2019 Sep;16(9):918-924. doi: 10.1038/s41592-019-0515-7. Epub 2019 Aug 26. PMID: 31451766.  




In [ ]:
#@title Fix OpenCL if needed in Google Colab
import sys

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !sudo apt-get update -qq
    !sudo apt-get purge -qq *nvidia* -y
    !sudo DEBIAN_FRONTEND=noninteractive apt-get install -qq nvidia-driver-530 -y
    exit(0)




## Load difraction limited image (only needed if you want to use the FRC and Decorrelation analysis on this image or if you want to perform the Error Map analysis)



In [ ]:
#@title Create image loader GUI for diffraction limited image
# Create a GUI
from nanopyx.core.utils.easy_gui import EasyGui
from nanopyx.core.utils.find_files import find_files

gui_data_df = EasyGui("Data Loader")

def on_button_load_data_clicked(b):
    clear_output()
    gui_data_df.show()
    global dataset_df
    # disable button
    gui_data_df["load_data"].disabled = True
    gui_data_df["load_data"].description = "Loading..."

    if gui_data_df["data_source"].value.startswith("Example dataset: "):
        dataset_name = gui_data_df["data_source"].value.replace(
            "Example dataset: ", "")
        dataset_df = EDM.get_ZipTiffIterator(dataset_name, as_ndarray=True)
        display(stackview.slice(dataset_df, continuous_update=True,
                                colormap=gui_data_df["cmaps"].value))
    else:
        dataset_df = skimage.io.imread(gui_data_df["data_source"].value)
        display(stackview.slice(dataset_df, continuous_update=True,
                                colormap=gui_data_df["cmaps"].value))
    
    # enable button
    gui_data_df["load_data"].disabled = False
    gui_data_df["load_data"].description = "Load data"
    gui_data_df.save_settings()

gui_data_df.add_label("Select data to use:")
gui_data_df.add_dropdown("data_source", options=image_files,
                 value="Example dataset: "+example_datasets[4], remember_value=True)
gui_data_df.add_dropdown("cmaps", description="Colormap:",
                      options=sorted(list(mpl.colormaps)),
                      value="viridis", remember_value=True)
gui_data_df.add_button("load_data", description="Load data")
gui_data_df["load_data"].on_click(on_button_load_data_clicked)
gui_data_df.show()




## Load super-resolved image



In [ ]:
#@title Create image loader GUI for SR image
# Create a GUI
from nanopyx.core.utils.easy_gui import EasyGui
from nanopyx.core.utils.find_files import find_files

gui_data_sr = EasyGui("Data Loader")

def on_button_load_data_clicked(b):
    clear_output()
    gui_data_sr.show()
    global dataset_sr
    # disable button
    gui_data_sr["load_data"].disabled = True
    gui_data_sr["load_data"].description = "Loading..."

    if gui_data_sr["data_source"].value.startswith("Example dataset: "):
        dataset_name = gui_data_sr["data_source"].value.replace(
            "Example dataset: ", "")
        dataset_sr = EDM.get_ZipTiffIterator(dataset_name, as_ndarray=True)
        display(stackview.slice(dataset_sr, continuous_update=True,
                                colormap=gui_data_sr["cmaps"].value))
    else:
        dataset_sr = skimage.io.imread(gui_data_sr["data_source"].value)
        display(stackview.slice(dataset_sr, continuous_update=True,
                                colormap=gui_data_sr["cmaps"].value))
    
    # enable button
    gui_data_sr["load_data"].disabled = False
    gui_data_sr["load_data"].description = "Load data"
    gui_data_sr.save_settings()

gui_data_sr.add_label("Select data to use:")
gui_data_sr.add_dropdown("data_source", options=image_files,
                 value="Example dataset: "+example_datasets[4], remember_value=True)
gui_data_sr.add_dropdown("cmaps", description="Colormap:",
                      options=sorted(list(mpl.colormaps)),
                      value="viridis", remember_value=True)
gui_data_sr.add_button("load_data", description="Load data")
gui_data_sr["load_data"].on_click(on_button_load_data_clicked)
gui_data_sr.show()




## Calculate Error Map



In [ ]:
#@title Create Error Map GUI
gui_error = EasyGui("Error")

import numpy as np
from matplotlib import pyplot as plt
from nanopyx.core.transform.new_error_map import ErrorMap

def run_error(b):
    clear_output()
    gui_error.show()
    gui_error.save_settings()
    gui_error["run"].disabled = True
    gui_error["run"].description = "Calculating..."
    global error_map
    error_map = ErrorMap()
    error_map.optimise(np.mean(dataset_df, axis=0), np.mean(dataset_sr[0], axis=0))
    errormap = np.array(error_map.imRSE)
    gui_error["run"].disabled = False
    gui_error["run"].description = "Calculate"
    print("RSE: ", error_map.getRSE())
    print("RSP: ", error_map.getRSP())
    plt.imshow(errormap)
    plt.axis("off")
    plt.show()

gui_error.add_button("run", description="Calculate")
gui_error["run"].on_click(run_error)
gui_error.show()




# Calculate FRC of the diffraction limited image
# FRC Parameters:

- **Pixel Size:** Pixel size of the image. Used to calculte resolution values.
- **Units:** Pixel size units.
- **First/Second Frame:** As FRC is calculated between two frames of the same image stack, these parameters determines which two frames are used for the calculation.




In [ ]:
#@title create FRC GUI for original image
gui_frc_df = EasyGui("FRC")

import numpy as np
from nanopyx.core.analysis.frc import FIRECalculator

def run_frc(b):
    clear_output()
    gui_frc_df.show()
    gui_frc_df.save_settings()
    pixel_size = gui_frc_df["pixel_size"].value
    units = gui_frc_df["units"].value
    first_frame = gui_frc_df["first_frame"].value
    second_frame = gui_frc_df["second_frame"].value
    gui_frc_df["run"].disabled = True
    gui_frc_df["run"].description = "Calculating..."
    global frc_calculator_raw
    frc_calculator_raw = FIRECalculator(pixel_size=pixel_size, units=units)
    frc_calculator_raw.calculate_fire_number(dataset_df[first_frame], dataset_df[second_frame])
    gui_frc_df["run"].disabled = False
    gui_frc_df["run"].description = "Calculate"
    plot = frc_calculator_raw.plot_frc_curve()
    if gui_frc_df["save"].value:
        if own_data_df:
            path = gui_data["upload"].selected_path
            name = gui_data["upload"].selected_filename.split(".")[0]
            tiff.imwrite(path + os.sep + name + "_original_FRC.tif", plot)
        else:
            name = gui_data["data_source"].value.replace("Example dataset: ", "")
            tiff.imwrite(name + "_FRC_df.tif", plot)
    plt.imshow(plot)
    plt.axis("off")
    plt.show()
    
gui_frc_df.add_int_slider("pixel_size", description="Pixel Size:", min=0.01, max=1000, value=100, remember_value=True)
gui_frc_df.add_dropdown("units", description="Units: ", options=["nm", "um", "mm"], value="nm")
gui_frc_df.add_int_slider("first_frame", description="First Frame:", min=0, max=dataset_df[0].shape[0]-1, value=0)
gui_frc_df.add_int_slider ("second_frame", description="Second Frame:", min=0, max=dataset_df[0].shape[0]-1, value=1)
gui_frc_df.add_checkbox("save", description="Save Output", value=True)
gui_frc_df.add_button("run", description="Calculate")
gui_frc_df["run"].on_click(run_frc)
gui_frc_df.show()




# Calculate FRC of the SR image
# FRC Parameters:

- **Pixel Size:** Pixel size of the image. Used to calculte resolution values.
- **Units:** Pixel size units.
- **First/Second Frame:** As FRC is calculated between two frames of the same image stack, these parameters determines which two frames are used for the calculation.




In [ ]:
#@title create FRC GUI for SR image
gui_frc_sr = EasyGui("FRC")

import numpy as np
from nanopyx.core.analysis.frc import FIRECalculator

def run_frc(b):
    clear_output()
    gui_frc_sr.show()
    gui_frc_sr.save_settings()
    pixel_size = gui_frc_sr["pixel_size"].value
    units = gui_frc_sr["units"].value
    first_frame = gui_frc_sr["first_frame"].value
    second_frame = gui_frc_sr["second_frame"].value
    gui_frc_sr["run"].disabled = True
    gui_frc_sr["run"].description = "Calculating..."
    global frc_calculator_raw
    frc_calculator_raw = FIRECalculator(pixel_size=pixel_size, units=units)
    frc_calculator_raw.calculate_fire_number(dataset_sr[first_frame], dataset_sr[second_frame])
    gui_frc_sr["run"].disabled = False
    gui_frc_sr["run"].description = "Calculate"
    plot = frc_calculator_raw.plot_frc_curve()
    if gui_frc_sr["save"].value:
        if own_data_df:
            path = gui_data["upload"].selected_path
            name = gui_data["upload"].selected_filename.split(".")[0]
            tiff.imwrite(path + os.sep + name + "_original_FRC.tif", plot)
        else:
            name = gui_data["data_source"].value.replace("Example dataset: ", "")
            tiff.imwrite(name + "_FRC_df.tif", plot)
    plt.imshow(plot)
    plt.axis("off")
    plt.show()
    
gui_frc_sr.add_int_slider("pixel_size", description="Pixel Size:", min=0.01, max=1000, value=100, remember_value=True)
gui_frc_sr.add_dropdown("units", description="Units: ", options=["nm", "um", "mm"], value="nm")
gui_frc_sr.add_int_slider("first_frame", description="First Frame:", min=0, max=dataset_df[0].shape[0]-1, value=0)
gui_frc_sr.add_int_slider ("second_frame", description="Second Frame:", min=0, max=dataset_df[0].shape[0]-1, value=1)
gui_frc_sr.add_checkbox("save", description="Save Output", value=True)
gui_frc_sr.add_button("run", description="Calculate")
gui_frc_sr["run"].on_click(run_frc)
gui_frc_sr.show()




# Calculate Decorrelation analysis of diffraction limited image
# Image Decorrelation Analysis Parameters:

- **Pixel Size:** Pixel size of the image. Used to calculte resolution values.
- **Units:** Pixel size units.
- **Frame:** Frame to be used for decorrelation analysis
- **Radius Min/Max:** Resolution calculation by Decorrelation Analysis is performed in the frequency space. These parameters define the range of radii to be used in the calculation. 




In [ ]:
#@title Create Decorr GUI for diffraction limited image
gui_decorr_df = EasyGui("DecorrAnalysis")

from nanopyx.core.analysis.decorr import DecorrAnalysis

def run_decorr(b):
    clear_output()
    gui_decorr_df.show()
    gui_decorr_df.save_settings()
    pixel_size = gui_decorr_df["pixel_size"].value
    units = gui_decorr_df["units"].value
    rmin = gui_decorr_df["rmin"].value
    rmax = gui_decorr_df["rmax"].value
    gui_decorr_df["run"].disabled = True
    gui_decorr_df["run"].description = "Calculating..."
    global decorr_calculator_raw
    decorr_calculator_raw = DecorrAnalysis(pixel_size=pixel_size, units=units, rmin=rmin, rmax=rmax)
    decorr_calculator_raw.run_analysis(dataset_df, axis=0))
    gui_decorr_df["run"].disabled = False
    gui_decorr_df["run"].description = "Calculate"
    plt.imshow(decorr_calculator_raw.plot_results())
    plt.axis("off")
    plt.show()
    
gui_decorr_df.add_int_slider("pixel_size", description="Pixel Size:", min=0.01, max=1000, value=100, remember_value=True)
gui_decorr_df.add_dropdown("units", description="Units: ", options=["nm", "um", "mm"], value="nm")
gui_decorr_df.add_int_slider("frame", description="Frame:", min=0, max=dataset_df[0].shape[0]-1, value=0)
gui_decorr_df.add_float_slider("rmin", description="Radius Min:", min=0.0, max=0.5, value=0.0)
gui_decorr_df.add_float_slider("rmax", description="Radius Max:", min=0.5, max=1.0, value=1.0)
gui_decorr_df.add_button("run", description="Calculate")
gui_decorr_df["run"].on_click(run_decorr)
gui_decorr_df.show()




# Calculate Decorrelation analysis of SR image
# Image Decorrelation Analysis Parameters:

- **Pixel Size:** Pixel size of the image. Used to calculte resolution values.
- **Units:** Pixel size units.
- **Frame:** Frame to be used for decorrelation analysis
- **Radius Min/Max:** Resolution calculation by Decorrelation Analysis is performed in the frequency space. These parameters define the range of radii to be used in the calculation. 




In [ ]:
#@title Create Decorr GUI for SR image
gui_decorr_sr = EasyGui("DecorrAnalysis")

from nanopyx.core.analysis.decorr import DecorrAnalysis

def run_decorr(b):
    clear_output()
    gui_decorr_sr.show()
    gui_decorr_sr.save_settings()
    pixel_size = gui_decorr_sr["pixel_size"].value
    units = gui_decorr_sr["units"].value
    rmin = gui_decorr_sr["rmin"].value
    rmax = gui_decorr_sr["rmax"].value
    gui_decorr_sr["run"].disabled = True
    gui_decorr_sr["run"].description = "Calculating..."
    global decorr_calculator_raw
    decorr_calculator_raw = DecorrAnalysis(pixel_size=pixel_size, units=units, rmin=rmin, rmax=rmax)
    decorr_calculator_raw.run_analysis(dataset_sr, axis=0))
    gui_decorr_sr["run"].disabled = False
    gui_decorr_sr["run"].description = "Calculate"
    plt.imshow(decorr_calculator_raw.plot_results())
    plt.axis("off")
    plt.show()
    
gui_decorr_sr.add_int_slider("pixel_size", description="Pixel Size:", min=0.01, max=1000, value=100, remember_value=True)
gui_decorr_sr.add_dropdown("units", description="Units: ", options=["nm", "um", "mm"], value="nm")
gui_decorr_sr.add_int_slider("frame", description="Frame:", min=0, max=dataset_sr[0].shape[0]-1, value=0)
gui_decorr_sr.add_float_slider("rmin", description="Radius Min:", min=0.0, max=0.5, value=0.0)
gui_decorr_sr.add_float_slider("rmax", description="Radius Max:", min=0.5, max=1.0, value=1.0)
gui_decorr_sr.add_button("run", description="Calculate")
gui_decorr_sr["run"].on_click(run_decorr)
gui_decorr_sr.show()

